## EESREP on AB case : 2050 mix supplying a balanced electric grid

After the serie of files "Hands-on EESREP", this code aims at reproducing the AB case studied by Itésé on RTE's ANTARES.

A climatic year is chosen between 1982 (1st set) and 2016 (35th and last set).

The OPEX data used here is defined in the MIX2050_A-B.R file provided by Itésé.

The model's structure can be exemplified as follows:

                                             | -->  load_A
                      flexible_Nuke 0-39 --> |
                             Solar PV A ---> |
                         Onshore Wind A ---> |
                        Offshore Wind A ---> |
            fuel_CCGT --> cluster_CCGT_A --> |
                demand-side management A --> |
                                             | --> spilled_energy_A
                                             |
                                             | <-----> Step_A <-----> |
                                             |                        | <---> Reservoir_A
                                             | <---  Turbines_A <---  |
                                             |
                                             | <--------------|
                                             |                |
                                             | ---->|    Interco_BA
                                                    |         |
                                               Interco_AB     |
                                                    |         |
                                             | <----|         |
                                             |                |
                                             | -------------->|
                                             |
                                             | <-----> Step_B <-----> |
                                             |                        | <---> Reservoir_B
                                             | <---  Turbines_B <---  |
                                             |
                                             | -->  load_B
                             Solar PV B ---> |
                         Onshore Wind B ---> |
                        Offshore Wind B ---> |
            fuel_CCGT --> cluster_CCGT_B --> |
                demand-side management B --> |
                                             | --> spilled_energy_B
Reste à faire :
- Solaire
- Eolien
- Stockage

### Imports

In [1]:
import math
import functools

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from eesrep import Eesrep
from eesrep.components.bus import GenericBus
from eesrep.components.generic_component import GenericComponent
from eesrep.components.storage import GenericStorage
from eesrep.components.dam import Dam
from eesrep.components.converter import Cluster, Converter
from eesrep.components.sink_source import FatalSource, FatalSink, Source, Sink
from eesrep.eesrep_enum import TimeSerieType, SolverOption
from eesrep.solver_interface.generic_interface import GenericInterface
from eesrep.eesrep_io import ComponentIO

from FlexibleNPP_component import FlexibleNPP

In [2]:
try:
    from compute_K_factor.cython_credit_K import credit_K_interprocessing
    print("Loaded cython credit K interprocessing function")

except ImportError as e:
    from compute_K_factor.credit_K import credit_K_interprocessing
    print("Loaded python credit K interprocessing function, for better performances, compile the cython version, see compute_K_factor/README.md")

Loaded cython credit K interprocessing function


### Simulation parameters

In [3]:
global_time_step = 1                # Time step size (hours)
global_time_shift = 50              # Shift between the beginning of two sucessing horizon (time steps)
global_future_size = 100            # Duration of each horizon (time steps)
global_horizon_count = 175           # Number of horizons to solve (horizon)

global_step_count = (global_horizon_count - 1) * global_time_shift + global_future_size     # Calculates the total number of time steps
#print("Nombre de pas de simulation :", global_step_count)                                  # Prints the previous value if needed

load_number = 24
Translation_A = True
Translation_B = True

# True activates the spilled energy virtual sink and demand-side management virtual source
global_bool_virtual_balancing = True

## Model definition

### User-defined inputs

#### Load

In [4]:
if Translation_A:
    file_A = "loadTranslation_a.txt"
else:
    file_A = "load_a.txt"
# end if    

if Translation_B:
    file_B = "loadTranslation_b.txt"
else:
    file_B = "load_b.txt"
# end if

path_load_a = "./Creation_JDD_UseCaseManoeuvrant/Donnees/load/series/"+file_A
path_load_b = "./Creation_JDD_UseCaseManoeuvrant/Donnees/load/series/"+file_B

load_a_df = pd.read_csv(path_load_a, sep="\t", header=None)
load_a_df.columns = [i+1 for i in range(len(load_a_df.columns))]
load_a = pd.concat([pd.DataFrame(load_a_df.index), load_a_df[load_number]], axis=1)
load_a.columns = ["time", "value"]

load_b_df = pd.read_csv(path_load_b, sep="\t", header=None)
load_b_df.columns = [i+1 for i in range(len(load_b_df.columns))]
load_b = pd.concat([pd.DataFrame(load_b_df.index), load_b_df[load_number]], axis=1)
load_b.columns = ["time", "value"]


#### Nuclear power plant with ramping constraints

In [5]:
### NPP caracteristics (most reference values found in ITESE A-B use case unless otherwise stated)
NPP_cycle_duration_fpd = 400            # Duration of the irradiation campaign in full power equivalent days : average on French nuclear fleet
NPP_RS_duration = 26                    # Standard duration of the refueling stop
NPP_efficiency = 0.33                   # Fuel to electricity ratio, no cogeneration considered
NPP_average_lf = 0.81                   # Average load factor during a campaign : historical data on French nuclear fleet
NPP_P_max = 1000.
NPP_P_min_rel = 0.2                     # Minimum relative output before shutdown
NPP_fuel_weight = 0.002 * 264 * 185     # Weight of the fuel in core (tons) : 2kg * 264 fuel rods * 185 assemblies (average on French nuclear fleet)
NPP_duration_on = 12                    # Minimum time enabled after turning on
NPP_duration_off = 16                   # Minimum time disabled after turning off
NPP_ramp_up = 0.03                      # Maximum ramping up rate after Cold Shutdown and refueling/repositioning stops
NPP_ramp_down = 1                       # Maximum ramping down rate

### Clusterized NPP caracteristics
Flex_nuke_n_A = 39                      # Number of NPP in zone A
Flex_nuke_n_B = 0                       # Number of NPP in zone B

# Set end-date of the refuelling stops of flexible units, in days after date 0.
Units_dates_RS_A = [0 for i in range(Flex_nuke_n_A)] # day of simulation when the refuelling stop ends
Units_dates_RS_B = [0 for i in range(Flex_nuke_n_B)]

# Set the mode of the flexible units, "Base" or "Load-following"
ELPO_mode_A = ["Load-following" for i in range(Flex_nuke_n_A)]
ELPO_mode_B = ["Load-following" for i in range(Flex_nuke_n_B)]

### Economic value
NPP_OPEX = 14.5                         # Cost of energy (€/MWh)
NPP_turn_on_price = 28. * NPP_P_max     # Cost of switching on (€/MW) multiplied by maximum power
NPP_FOPEX = 0.01                        # Default to disable cluster when useless (€/h)

### Individual nuclear power plant behaviour constraints
# True activates ramping constraints
global_NPP_bool_ramps = False
# True activates ELPO credit constraints
global_NPP_bool_creditELPO = True
dict_K0 = {"Base" : 200, "Load-following" : 100}
dict_A_i = {"0 to 2000 MWj/t" : {"27 %Pn to 85 %Pn" : 4.9, "85 %Pn to 92 %Pn" : 2.8},
    "2000 to 6000 MWj/t" : {"27 %Pn to 85 %Pn" : 6.1, "85 %Pn to 92 %Pn" : 2.3},
    "6000 MWj/t to end" : {"27 %Pn to 50 %Pn" : 4.1, "50 %Pn to 92 %Pn" : 6.3}
    }
conservative_A_i = max([max(dict_A_i[key].values()) for key in dict_A_i.keys()])
dict_B_j = {"Base" : {"0 to 100" : [8, -0.07], "100 to 200" : [1, 0], "200" : [0, 0]},
            "Load-following" : {"0 to 80" : [4.5, -0.05], "80 to 150" : [0.5, 0], "150" : [0, 0]}
            }
dict_K_max = {"Base" : 200, "Load-following" : 150}
# True activates minimum steady duration after transitions
global_NPP_bool_duration = True
# True activates the full power equivalent days monitoring
global_NPP_bool_fpd = True
# True activates a burn-up approximation on endogenous production history
global_NPP_endogenous_burn_up_approx = True


#### CCGT cluster

In [6]:
### CCGT caracteristics (reference values found in ITESE A-B use case)
CCGT_efficiency = 0.57                      # Fuel to electricity ratio, no cogeneration considered
CCGT_P_max = 100                            # Itésé data
CCGT_P_min_rel = 0.4                        # Minimum relative output before shutdown
CCGT_duration_on = 2                        # Minimum time enabled after turning on
CCGT_duration_off = 2                       # Minimum time disabled after turning off

### cluster_CCGT caracteristics
N2A = 18                                    # Number of CCGTs in the cluster in the zone A
N2B = 75                                    # Number of CCGTs in the cluster in the zone B

cluster_units = {"cluster_CCGT_A" : N2A,
                 "cluster_CCGT_B" : N2B}

### Economic value
CCGT_OPEX = 210.                            # Cost of energy (€/MWh)
CCGT_turn_on_price = 278. * CCGT_P_max      # Cost of switching on (€/MW) multiplied by maximum power
CCGT_FOPEX = 0.01                           # Default to disable cluster when useless (€/h)

#### Demand-side management : unsupplied energy

In [7]:
### equivalent virtual powerplant caracteristics (reference values found in ITESE A-B use case)
DSM_efficiency = 1.                     # Fuel to electricity ratio, no cogeneration considered
DSM_P_max = None
DSM_P_min_rel = 0.                      # Minimum relative output before shutdown
DSM_duration_on = 6.                    # Minimum time enabled after turning on
DSM_duration_off = 1.                   # Minimum time disabled after turning off
DSM_turn_on_price = 0.                  # Cost of switching on (€/MW)

### Economic virtual value
DSM_OPEX = 15000                        # Cost of energy (€/MWh)
DSM_turn_on_price = 0.                  # Cost of switching on (€/MW)

#### Spilled energy

In [8]:
### equivalent virtual powerplant caracteristics (reference values found in ITESE A-B use case)
SE_efficiency = 1.                      # Fuel to electricity ratio, no cogeneration considered
SE_P_max = None                         # Arbitrary
SE_P_min_rel = 0.                       # Minimum relative output before shutdown
SE_duration_on = 1.                     # Minimum time enabled after turning on
SE_duration_off = 1.                    # Minimum time disabled after turning off
SE_turn_on_price = 0.                   # Cost of switching on (€/MW)

### Economic virtual value
SE_OPEX = 1                             # Cost of energy (€/MWh)
SE_turn_on_price = 0.                   # Cost of switching on (€/MW)

#### Fuels

In [9]:
fuel_Nuke_price = NPP_OPEX * NPP_efficiency     # Default: back-calculation of the cost of nuclear fuel with NPP OPEX
fuel_Nuke_lb = 0.                               # Minimum consumption rate if relevant (contracts), default: 0
fuel_Nuke_ub = None                             # Maximum consumption rate if relevant (supply limits), default: None

fuel_CCGT_price = CCGT_OPEX * CCGT_efficiency   # Default: back-calculation of the cost of CCGT fuel with CCGT OPEX
fuel_CCGT_lb = 0.                               # Minimum consumption rate if relevant (contracts), default: 0
fuel_CCGT_ub = None                             # Maximum consumption rate if relevant (supply limits), default: None

fuel_DSM_price = DSM_OPEX * DSM_efficiency      # Default: back-calculation of the cost of DSM fuel with DSM OPEX
fuel_DSM_lb = 0.                                # Minimum consumption rate
fuel_DSM_ub = None                              # Maximum consumption rate

#### Interconnection AB

In [10]:
AB_capacity = 6000                         # (MW)
AB_OPEX = 2                                # (€/MWh)

BA_capacity = 6000                         # (MW)
BA_OPEX = 2                                # (€/MWh)

#### Storage

##### Dams (irreversible storage)

In [11]:
Reservoir_A = 10100000                      # (MWh)
Reservoir_B = 917500

Turbine_A = 21600                           # (MW)
Turbine_B = 5300

##### STEPs (reversible storage)

In [12]:
Reservoir_STEP_A = 218500                   # (MWh)
Reservoir_STEP_B = 834000

Turbine_STEP_A = 10000                      # (MW)
Turbine_STEP_B = 15000

Pompes_STEP_A = 10000                       # (MW)
Pompes_STEP_B = 15000

Efficiency_STEP_A = 0.8
Efficiency_STEP_B = 0.8

#### Renewables

In [13]:
Capacity_solarPV_A = 90000 
Capacity_windOff_A = 36100
Capacity_windOn_A = 51500

Capacity_solarPV_B = 196600
Capacity_windOff_B = 79100
Capacity_windOn_B = 193000

Renewables_capacities = {"Solar_PV_A" : Capacity_solarPV_A,
                         "WindOff_A" : Capacity_windOff_A,
                         "WindOn_A" : Capacity_windOn_A,
                         "Solar_PV_B" : Capacity_solarPV_B,
                         "WindOff_B" : Capacity_windOff_B,
                         "WindOn_B" : Capacity_windOn_B}

##### File path #####
path_solarPV_A = "./Creation_JDD_UseCaseManoeuvrant/Donnees/renewables/series/a/solar pv/series.txt"
path_windOff_A = "./Creation_JDD_UseCaseManoeuvrant/Donnees/renewables/series/a/wind offshore/series.txt"
path_windOn_A = "./Creation_JDD_UseCaseManoeuvrant/Donnees/renewables/series/a/wind onshore/series.txt"
path_hydroROR_A = "./Creation_JDD_UseCaseManoeuvrant/Donnees/hydro/series/a/ror.txt"
path_hydroMOD_A = "./Creation_JDD_UseCaseManoeuvrant/Donnees/hydro/series/a/mod.txt"
path_hydroSTO_A = "./Creation_JDD_UseCaseManoeuvrant/Donnees/hydro/common/capacity/reservoir_a.txt"

path_solarPV_B = "./Creation_JDD_UseCaseManoeuvrant/Donnees/renewables/series/b/solar pv/series.txt"
path_windOff_B = "./Creation_JDD_UseCaseManoeuvrant/Donnees/renewables/series/b/wind offshore/series.txt"
path_windOn_B = "./Creation_JDD_UseCaseManoeuvrant/Donnees/renewables/series/b/wind onshore/series.txt"
path_hydroROR_B = "./Creation_JDD_UseCaseManoeuvrant/Donnees/hydro/series/b/ror.txt"
path_hydroMOD_B = "./Creation_JDD_UseCaseManoeuvrant/Donnees/hydro/series/b/mod.txt"
path_hydroSTO_B = "./Creation_JDD_UseCaseManoeuvrant/Donnees/hydro/common/capacity/reservoir_b.txt"

##### .txt read into Pandas Dataframe #####
production_solarPV_A_df = pd.read_csv(path_solarPV_A, sep="\t", header=None) * Capacity_solarPV_A
production_windOff_A_df = pd.read_csv(path_windOff_A, sep="\t", header=None) * Capacity_windOff_A
production_windOn_A_df = pd.read_csv(path_windOn_A, sep="\t", header=None) * Capacity_windOn_A
production_hydroROR_A_df = pd.read_csv(path_hydroROR_A, sep="\t", header=None)
production_hydroMOD_A_df = pd.read_csv(path_hydroMOD_A, sep="\t", header=None)
production_hydroSTO_A_df = pd.read_csv(path_hydroSTO_A, sep="\t", header=None)

production_solarPV_B_df = pd.read_csv(path_solarPV_B, sep="\t", header=None) * Capacity_solarPV_B
production_windOff_B_df = pd.read_csv(path_windOff_B, sep="\t", header=None) * Capacity_windOff_B
production_windOn_B_df = pd.read_csv(path_windOn_B, sep="\t", header=None) * Capacity_windOn_B
production_hydroROR_B_df = pd.read_csv(path_hydroROR_B, sep="\t", header=None)
production_hydroMOD_B_df = pd.read_csv(path_hydroMOD_B, sep="\t", header=None)
production_hydroSTO_B_df = pd.read_csv(path_hydroSTO_B, sep="\t", header=None)

##### Pandas Dataframe formatting #####
production_solarPV_A_df.columns = [i+1 for i in range(len(production_solarPV_A_df.columns))]
production_solarPV_A = pd.concat([pd.DataFrame(production_solarPV_A_df.index), production_solarPV_A_df[load_number]], axis=1)
production_solarPV_A.columns = ["time", "value"]
production_windOff_A_df.columns = [i+1 for i in range(len(production_windOff_A_df.columns))]
production_windOff_A = pd.concat([pd.DataFrame(production_windOff_A_df.index), production_windOff_A_df[load_number]], axis=1)
production_windOff_A.columns = ["time", "value"]
production_windOn_A_df.columns = [i+1 for i in range(len(production_windOn_A_df.columns))]
production_windOn_A = pd.concat([pd.DataFrame(production_windOn_A_df.index), production_windOn_A_df[load_number]], axis=1)
production_windOn_A.columns = ["time", "value"]
production_hydroROR_A_df.columns = [i+1 for i in range(len(production_hydroROR_A_df.columns))]
production_hydroROR_A = pd.concat([pd.DataFrame(production_hydroROR_A_df.index), production_hydroROR_A_df[load_number]], axis=1)
production_hydroROR_A.columns = ["time", "value"]
production_hydroMOD_A_df.columns = [i+1 for i in range(len(production_hydroMOD_A_df.columns))]
production_hydroMOD_A = pd.concat([pd.DataFrame(production_hydroMOD_A_df.index), production_hydroMOD_A_df[load_number]], axis=1)
production_hydroMOD_A.columns = ["time", "value"]
production_hydroSTO_A_df.columns = ["min", "average", "max"]
production_hydroSTO_A_min = pd.concat([pd.DataFrame(production_hydroSTO_A_df.index), production_hydroSTO_A_df["min"]], axis=1)
production_hydroSTO_A_avg = pd.concat([pd.DataFrame(production_hydroSTO_A_df.index), production_hydroSTO_A_df["average"]], axis=1)
production_hydroSTO_A_max = pd.concat([pd.DataFrame(production_hydroSTO_A_df.index), production_hydroSTO_A_df["max"]], axis=1)
production_hydroSTO_A_min.columns = ["time", "value"]
production_hydroSTO_A_avg.columns = ["time", "value"]
production_hydroSTO_A_max.columns = ["time", "value"]

production_hydroMOD_A["time"] = pd.to_datetime(pd.to_datetime("1981-01-01 00:00:00") + pd.DateOffset(years=load_number) + pd.to_timedelta(production_hydroMOD_A["time"], unit="days"))
production_hydroMOD_A.set_index(production_hydroMOD_A["time"], inplace = True)
production_hydroMOD_A.drop(labels=["time"], axis="columns", inplace = True)
production_hydroMOD_A = production_hydroMOD_A.resample('1h').asfreq() / 24
production_hydroMOD_A = production_hydroMOD_A.ffill()
production_hydroMOD_A.reset_index(inplace=True)
production_hydroMOD_A["time"] = production_hydroMOD_A.index
production_hydroMOD_A = production_hydroMOD_A.reindex(labels=["time", "value"], axis=1)
# production_hydroMOD_A["value"] = production_hydroMOD_A["value"] / Reservoir_A

production_solarPV_B_df.columns = [i+1 for i in range(len(production_solarPV_B_df.columns))]
production_solarPV_B = pd.concat([pd.DataFrame(production_solarPV_B_df.index), production_solarPV_B_df[load_number]], axis=1)
production_solarPV_B.columns = ["time", "value"]
production_windOff_B_df.columns = [i+1 for i in range(len(production_windOff_B_df.columns))]
production_windOff_B = pd.concat([pd.DataFrame(production_windOff_B_df.index), production_windOff_B_df[load_number]], axis=1)
production_windOff_B.columns = ["time", "value"]
production_windOn_B_df.columns = [i+1 for i in range(len(production_windOn_B_df.columns))]
production_windOn_B = pd.concat([pd.DataFrame(production_windOn_B_df.index), production_windOn_B_df[load_number]], axis=1)
production_windOn_B.columns = ["time", "value"]
production_hydroROR_B_df.columns = [i+1 for i in range(len(production_hydroROR_B_df.columns))]
production_hydroROR_B = pd.concat([pd.DataFrame(production_hydroROR_B_df.index), production_hydroROR_B_df[load_number]], axis=1)
production_hydroROR_B.columns = ["time", "value"]
production_hydroMOD_B_df.columns = [i+1 for i in range(len(production_hydroMOD_B_df.columns))]
production_hydroMOD_B = pd.concat([pd.DataFrame(production_hydroMOD_B_df.index), production_hydroMOD_B_df[load_number]], axis=1)
production_hydroMOD_B.columns = ["time", "value"]
production_hydroSTO_B_df.columns = ["min", "average", "max"]
production_hydroSTO_B_min = pd.concat([pd.DataFrame(production_hydroSTO_B_df.index), production_hydroSTO_B_df["min"]], axis=1)
production_hydroSTO_B_avg = pd.concat([pd.DataFrame(production_hydroSTO_B_df.index), production_hydroSTO_B_df["average"]], axis=1)
production_hydroSTO_B_max = pd.concat([pd.DataFrame(production_hydroSTO_B_df.index), production_hydroSTO_B_df["max"]], axis=1)
production_hydroSTO_B_min.columns = ["time", "value"]
production_hydroSTO_B_avg.columns = ["time", "value"]
production_hydroSTO_B_max.columns = ["time", "value"]

production_hydroMOD_B["time"] = pd.to_datetime(pd.to_datetime("1981-01-01 00:00:00") + pd.DateOffset(years=load_number) + pd.to_timedelta(production_hydroMOD_B["time"], unit="days"))
production_hydroMOD_B.set_index(production_hydroMOD_B["time"], drop = True, inplace = True)
production_hydroMOD_B.drop(labels=["time"], axis="columns", inplace = True)
production_hydroMOD_B = production_hydroMOD_B.resample('1h').asfreq() / 24
production_hydroMOD_B = production_hydroMOD_B.ffill()
production_hydroMOD_B.reset_index(inplace=True)
production_hydroMOD_B["time"] = production_hydroMOD_B.index
production_hydroMOD_B = production_hydroMOD_B.reindex(labels=["time", "value"], axis=1)
# production_hydroMOD_B["value"] = production_hydroMOD_B["value"] / Reservoir_B

print(production_hydroMOD_A)

      time     value
0        0  2726.250
1        1  2726.250
2        2  2726.250
3        3  2726.250
4        4  2726.250
...    ...       ...
8732  8732  1965.375
8733  8733  1965.375
8734  8734  1965.375
8735  8735  1965.375
8736  8736  1965.375

[8737 rows x 2 columns]


#### Boolean shortcuts calculation

In [14]:
### Energy mix configuration
# True instanciates a given number of nuclear individual powerplants
global_bool_Nuke_indiv = (Flex_nuke_n_A + Flex_nuke_n_B > 0)
# True instanciates nuclear clusterized powerplants
global_bool_Nuke_cluster = False
# True instanciates CCGT clusterized powerplants
global_bool_CCGT_cluster = (N2A + N2B > 0)
# True instanciates OCGT clusterized powerplants
global_bool_OCGT_cluster = False

global_bool_nuke = global_bool_Nuke_cluster + global_bool_Nuke_indiv                        # Calculates if nuclear fuel has to be instanciated
global_bool_gaz = global_bool_CCGT_cluster + global_bool_OCGT_cluster                       # Calculates if gaz fuel has to be instanciated

###   Model and components instantiation

The following blocks generates the EESREP object and each component.

In [15]:
model = Eesrep(interface="docplex")
model.define_time_range(time_step = global_time_step, 
                        time_shift = global_time_shift, 
                        future_size = global_future_size, 
                        horizon_count = global_horizon_count)

#### Bus instantiation

In [16]:
bus_elec_A = GenericBus("bus_elec_A")
model.add_component(bus_elec_A)

bus_elec_B = GenericBus("bus_elec_B")
model.add_component(bus_elec_B)

bus_nuke = GenericBus("bus_nuke")
model.add_component(bus_nuke)
    
bus_gas = GenericBus("bus_gas")
model.add_component(bus_gas)

bus_gas_fopex = GenericBus("bus_gas_fopex")
model.add_component(bus_gas_fopex)

bus_water = GenericBus("bus_water")
model.add_component(bus_water)

#### Load

In [17]:
load_A = FatalSink(name="load_A", 
                    sink_flow = load_a)
model.add_component(load_A)

model.plug_to_bus(io = load_A.power_in,
                  bus_io = bus_elec_A.output,
                  factor = 1., offset = 0.)

load_B = FatalSink(name="load_B", 
                    sink_flow = load_b)
model.add_component(load_B)

model.plug_to_bus(io = load_B.power_in,
                  bus_io = bus_elec_B.output,
                  factor = 1., offset = 0.)

#### Virtual balancing components : spilled energy and demand-side management

In [18]:
if global_bool_virtual_balancing:
    DSM_A = Source(name="Demand_side_Management_A",
                p_max = DSM_P_max,
                p_min = 0,
                price = fuel_DSM_price)
    DSM_B = Source(name="Demand_side_Management_B",
                p_max = DSM_P_max,
                p_min = 0,
                price = fuel_DSM_price)
    
    model.add_component(DSM_A)
    model.add_component(DSM_B)

    model.plug_to_bus(io = DSM_A.power_out,
                    bus_io = bus_elec_A.input,
                    factor = 1., offset = 0.)
    model.plug_to_bus(io = DSM_B.power_out,
                    bus_io = bus_elec_B.input,
                    factor = 1., offset = 0.)

    spilled_A = Sink(name="spilled_energy_A", 
                p_min = 0.,
                p_max = SE_P_max,
                price = SE_OPEX)
    spilled_B = Sink(name="spilled_energy_B", 
                p_min = 0.,
                p_max = SE_P_max,
                price = SE_OPEX)
    
    model.add_component(spilled_A)
    model.add_component(spilled_B)

    model.plug_to_bus(io = spilled_A.power_in,
                    bus_io = bus_elec_A.output,
                    factor = 1., offset = 0.)
    model.plug_to_bus(io = spilled_B.power_in,
                    bus_io = bus_elec_B.output,
                    factor = 1., offset = 0.)

#### Fuel source components

In [19]:
if global_bool_nuke:
    fuel_Nuke = Source(name="fuel_Nuke",
                    p_max = fuel_Nuke_ub,
                    p_min = fuel_Nuke_lb,
                    price = fuel_Nuke_price)
    
    model.add_component(fuel_Nuke)

    model.plug_to_bus(io = fuel_Nuke.power_out,
                    bus_io = bus_nuke.input,
                    factor = 1., offset = 0.)
# end if

if global_bool_gaz:
    fuel_CCGT = Source(name="fuel_CCGT",
                    p_max = fuel_CCGT_ub,
                    p_min = fuel_CCGT_lb,
                    price = fuel_CCGT_price)
    
    model.add_component(fuel_CCGT)

    model.plug_to_bus(io = fuel_CCGT.power_out,
                    bus_io = bus_gas.input,
                    factor = 1., offset = 0.)
# end if


#### Virtual OPEX calculation sinks

In [20]:
### Virtual sinks for fixed OPEX of power plants and clusters
if global_bool_CCGT_cluster:
    price_cluster_CCGT_on = Sink(name="price_cluster_CCGT_on", 
                            p_min=0.,
                            p_max=None,
                            price = CCGT_FOPEX)
    
    model.add_component(price_cluster_CCGT_on)

    model.plug_to_bus(io = price_cluster_CCGT_on.power_in,
                      bus_io = bus_gas_fopex.output,
                      factor = 1., offset = 0.)
# end if

#### Individual flexible power plant components

In [21]:
if global_bool_Nuke_indiv:
    Flexible_units = {}
        
    for i in range(Flex_nuke_n_A):
        name = "Flexible_Nuke_A_"+str(i)
        Flexible_units[name] = FlexibleNPP(name = name,
                                           fpd_init = 0,
                                           fpd_max = NPP_cycle_duration_fpd,
                                           endogenous_burn_up_approx = True,
                                           RS_duration = NPP_RS_duration,
                                           date_end_RS = Units_dates_RS_A[i],
                                           bool_fpd = True,
                                           efficiency = NPP_efficiency,
                                           average_lf = NPP_average_lf,
                                           p_max = NPP_P_max,
                                           duration_on = NPP_duration_on,
                                           duration_off = NPP_duration_off,
                                           fuel_weight= NPP_fuel_weight,
                                           opex = NPP_OPEX + i * 1e-7,
                                           turn_on_price = NPP_turn_on_price,
                                           ramp_up = NPP_ramp_up,
                                           ramp_down = NPP_ramp_down,
                                           bool_ramps = global_NPP_bool_ramps,
                                           creditELPOmax = dict_K_max[ELPO_mode_A[i]],
                                           bool_creditELPO = global_NPP_bool_creditELPO,
                                           ELPO_mode = ELPO_mode_A[i],
                                           K0 = dict_K0[ELPO_mode_A[i]],
                                           cons_A_i = conservative_A_i,
                                           bool_duration = global_NPP_bool_duration)
        model.add_component(Flexible_units[name])

        model.plug_to_bus(io = Flexible_units[name].electricity,
                    bus_io = bus_elec_A.input,
                    factor = 1., offset = 0.)
        model.plug_to_bus(io = Flexible_units[name].fuel,
                    bus_io = bus_nuke.output,
                    factor = 1. + i*1e-6, offset = 0.)
        model.plug_to_bus(io = Flexible_units[name].CS,
                    bus_io = bus_nuke.input,
                    factor = -1., offset = 1.)
    # end for

    for i in range(Flex_nuke_n_B):
        name = "Flexible_Nuke_B_"+str(i)
        Flexible_units[name] = FlexibleNPP(name = name,
                                           fpd_init = 0,
                                           fpd_max = NPP_cycle_duration_fpd,
                                           endogenous_burn_up_approx = True,
                                           RS_duration = NPP_RS_duration,
                                           date_end_RS = Units_dates_RS_B[i],
                                           bool_fpd = True,
                                           efficiency = NPP_efficiency,
                                           average_lf = NPP_average_lf,
                                           p_max = NPP_P_max,
                                           duration_on = NPP_duration_on,
                                           duration_off = NPP_duration_off,
                                           fuel_weight= NPP_fuel_weight,
                                           opex = NPP_OPEX,
                                           turn_on_price = NPP_turn_on_price,
                                           ramp_up = NPP_ramp_up,
                                           ramp_down = NPP_ramp_down,
                                           bool_ramps = global_NPP_bool_ramps,
                                           creditELPOmax = dict_K_max[ELPO_mode_B[i]],
                                           bool_creditELPO = global_NPP_bool_creditELPO,
                                           ELPO_mode = ELPO_mode_B[i],
                                           K0 = dict_K0[ELPO_mode_B[i]],
                                           cons_A_i = conservative_A_i,
                                           bool_duration = global_NPP_bool_duration)
        model.add_component(Flexible_units[name])

        model.plug_to_bus(io = Flexible_units[name].electricity,
                    bus_io = bus_elec_B.input,
                    factor = 1., offset = 0.)
        model.plug_to_bus(io = Flexible_units[name].fuel,
                    bus_io = bus_nuke.output,
                    factor = 1. + i*1e-6, offset = 0.)
        model.plug_to_bus(io = Flexible_units[name].CS,
                    bus_io = bus_nuke.input,
                    factor = -1., offset = 1.)
    # end for
    
    print(Flexible_units)
    model.set_post_processing(functools.partial(credit_K_interprocessing, flexible_units=Flexible_units, dict_K0=dict_K0, dict_A_i=dict_A_i, dict_B_j=dict_B_j))
# end if

{'Flexible_Nuke_A_0': <FlexibleNPP_component.FlexibleNPP object at 0x7fd58d72e310>, 'Flexible_Nuke_A_1': <FlexibleNPP_component.FlexibleNPP object at 0x7fd58d72e750>, 'Flexible_Nuke_A_2': <FlexibleNPP_component.FlexibleNPP object at 0x7fd58d72ebd0>, 'Flexible_Nuke_A_3': <FlexibleNPP_component.FlexibleNPP object at 0x7fd58d751090>, 'Flexible_Nuke_A_4': <FlexibleNPP_component.FlexibleNPP object at 0x7fd58d751590>, 'Flexible_Nuke_A_5': <FlexibleNPP_component.FlexibleNPP object at 0x7fd58d751a50>, 'Flexible_Nuke_A_6': <FlexibleNPP_component.FlexibleNPP object at 0x7fd58d751ed0>, 'Flexible_Nuke_A_7': <FlexibleNPP_component.FlexibleNPP object at 0x7fd58d725390>, 'Flexible_Nuke_A_8': <FlexibleNPP_component.FlexibleNPP object at 0x7fd58d751550>, 'Flexible_Nuke_A_9': <FlexibleNPP_component.FlexibleNPP object at 0x7fd58d751a10>, 'Flexible_Nuke_A_10': <FlexibleNPP_component.FlexibleNPP object at 0x7fd58d724090>, 'Flexible_Nuke_A_11': <FlexibleNPP_component.FlexibleNPP object at 0x7fd58d724510>, '

#### Clusterized power plants components

In [22]:
if global_bool_CCGT_cluster:
    cluster_CCGT_A = Cluster(name="cluster_CCGT_A",
                                    efficiency = CCGT_efficiency,
                                    p_max = CCGT_P_max,
                                    p_min = CCGT_P_min_rel * CCGT_P_max,
                                    n_machine_max = N2A,
                                    duration_on = CCGT_duration_on,
                                    duration_off = CCGT_duration_off,
                                    turn_on_price = CCGT_turn_on_price)
    
    cluster_CCGT_B = Cluster(name="cluster_CCGT_B",
                                    efficiency = CCGT_efficiency,
                                    p_max = CCGT_P_max,
                                    p_min = CCGT_P_min_rel * CCGT_P_max,
                                    n_machine_max = N2B,
                                    duration_on = CCGT_duration_on,
                                    duration_off = CCGT_duration_off,
                                    turn_on_price = CCGT_turn_on_price)
    
    model.add_component(cluster_CCGT_A)
    model.add_component(cluster_CCGT_B)

    model.plug_to_bus(io = cluster_CCGT_A.power_out,
                      bus_io = bus_elec_A.input,
                      factor = 1., offset = 0.)
    model.plug_to_bus(io = cluster_CCGT_A.power_in,
                      bus_io = bus_gas.output,
                      factor=1., offset=0.)
    model.plug_to_bus(io = cluster_CCGT_A.n_machine,
                      bus_io = bus_gas_fopex.input,
                      factor=1., offset=0.)
    model.plug_to_bus(io = cluster_CCGT_B.power_out,
                      bus_io = bus_elec_B.input,
                      factor = 1., offset = 0.)
    model.plug_to_bus(io = cluster_CCGT_B.power_in,
                      bus_io = bus_gas.output,
                      factor=1., offset=0.)
    model.plug_to_bus(io = cluster_CCGT_B.n_machine,
                      bus_io = bus_gas_fopex.input,
                      factor=1., offset=0.)
# end if

#### Interconnection

In [23]:
Interco_AB = Converter(name="Interco_AB",
                       efficiency=1,
                       p_min=0,
                       p_max=AB_capacity)

model.add_component(Interco_AB)
model.add_io_to_objective(Interco_AB.power_in, AB_OPEX)

model.plug_to_bus(io = Interco_AB.power_out,
                      bus_io = bus_elec_B.input,
                      factor = 1., offset = 0.)
model.plug_to_bus(io = Interco_AB.power_in,
                      bus_io = bus_elec_A.output,
                      factor = 1., offset = 0.)

Interco_BA = Converter(name="Interco_BA",
                       efficiency=1,
                       p_min=0,
                       p_max=BA_capacity)

model.add_component(Interco_BA)
model.add_io_to_objective(Interco_BA.power_in, BA_OPEX)

model.plug_to_bus(io = Interco_BA.power_out,
                      bus_io = bus_elec_A.input,
                      factor = 1., offset = 0.)
model.plug_to_bus(io = Interco_BA.power_in,
                      bus_io = bus_elec_B.output,
                      factor = 1., offset = 0.)

##### Dams (irreversible storage)

In [24]:
Dams_A = Dam(name="Dams_A",
                   efficiency=1,
                   p_min=0,
                   p_max=Turbine_A,
                   max_storage=Reservoir_A,
                   init_storage=0.555,
                   pump_max=0,
                   pump_efficiency=math.sqrt(0.8),
                   free_output=True,
                   power_input=False,
                   difference_from_limit_price=1,
                   difference_from_average_price=0.1,
                   water_inflow=production_hydroMOD_A,
                   variable_storage_min=production_hydroSTO_A_min,
                   variable_storage_average=production_hydroSTO_A_avg,
                   variable_storage_max=production_hydroSTO_A_max,
                   #???
                   )

Dams_B = Dam(name="Dams_B",
                   efficiency=1,
                   p_min=0,
                   p_max=Turbine_B,
                   max_storage=Reservoir_B,
                   init_storage=0.440,
                   pump_max=0,
                   pump_efficiency=math.sqrt(0.8),
                   free_output=True,
                   power_input=False,
                   difference_from_limit_price=1,
                   difference_from_average_price=0.1,
                   water_inflow=production_hydroMOD_B,
                   variable_storage_min=production_hydroSTO_B_min,
                   variable_storage_average=production_hydroSTO_B_avg,
                   variable_storage_max=production_hydroSTO_B_max,
                   )

model.add_component(Dams_A)
model.add_component(Dams_B)

model.plug_to_bus(io = Dams_A.power_out,
                      bus_io = bus_elec_A.input,
                      factor = 1., offset = 0.)
# model.plug_to_bus(io = Dams_A.power_free,
#                       bus_io = bus_water.input,
#                       factor = 1., offset = 0.)
model.plug_to_bus(io = Dams_B.power_out,
                      bus_io = bus_elec_B.input,
                      factor = 1., offset = 0.)
# model.plug_to_bus(io = Dams_B.power_free,
#                       bus_io = bus_water.input,
#                       factor = 1., offset = 0.)

# Water_sink = Sink(name="spilled_water_sink", 
#                   p_min = 0.,
#                   p_max = None,
#                   price = 0)

# model.add_component(Water_sink)

# model.plug_to_bus(io = Water_sink.power_in,
#                   bus_io = bus_water.output,
#                   factor = 1., offset = 0.)

##### STEPs (reversible storage)

In [25]:
STEPs_A = Dam(name="STEPs_A",
                   efficiency=math.sqrt(0.8),
                   p_min=0,
                   p_max=Turbine_STEP_A,
                   max_storage=Reservoir_STEP_A,
                   init_storage=0.5,
                   pump_max=Turbine_STEP_A,
                   pump_efficiency=math.sqrt(0.8),
                   free_output=False,
                   power_input=False,
                   difference_from_limit_price=1, #???
                   difference_from_average_price=1 #???
                   )
STEPs_B = Dam(name="STEPs_B",
                   efficiency=math.sqrt(0.8),
                   p_min=0,
                   p_max=Turbine_STEP_B,
                   max_storage=Reservoir_STEP_B,
                   init_storage=0.5,
                   pump_max=Turbine_STEP_B,
                   pump_efficiency=math.sqrt(0.8),
                   free_output=False,
                   power_input=False,
                   difference_from_limit_price=1, #???
                   difference_from_average_price=1 #???
                   )

model.add_component(STEPs_A)
model.add_component(STEPs_B)

model.plug_to_bus(io = STEPs_A.power_out,
                      bus_io = bus_elec_A.input,
                      factor = 1., offset = 0.)
model.plug_to_bus(io = STEPs_A.power_pump,
                      bus_io = bus_elec_A.output,
                      factor = 1., offset = 0.)
model.plug_to_bus(io = STEPs_B.power_out,
                      bus_io = bus_elec_B.input,
                      factor = 1., offset = 0.)
model.plug_to_bus(io = STEPs_B.power_pump,
                      bus_io = bus_elec_B.output,
                      factor = 1., offset = 0.)

#### Renewables

In [26]:
Solar_PV_A = FatalSource(name="Solar_PV_A",
                    source_flow = production_solarPV_A)
WindOff_A = FatalSource(name="WindOff_A",
                    source_flow = production_windOff_A)
WindOn_A = FatalSource(name="WindOn_A",
                    source_flow = production_windOn_A)
hydroROR_A = FatalSource(name="hydroROR_A",
                    source_flow = production_hydroROR_A)

Solar_PV_B = FatalSource(name="Solar_PV_B",
                    source_flow = production_solarPV_B)
WindOff_B = FatalSource(name="WindOff_B",
                    source_flow = production_windOff_B)
WindOn_B = FatalSource(name="WindOn_B",
                    source_flow = production_windOn_B)
hydroROR_B = FatalSource(name="hydroROR_B",
                    source_flow = production_hydroROR_B)
    
model.add_component(Solar_PV_A)
model.add_component(WindOff_A)
model.add_component(WindOn_A)
model.add_component(hydroROR_A)

model.add_component(Solar_PV_B)
model.add_component(WindOff_B)
model.add_component(WindOn_B)
model.add_component(hydroROR_B)

model.plug_to_bus(io = Solar_PV_A.power_out,
                    bus_io = bus_elec_A.input,
                    factor = 1., offset = 0.)
model.plug_to_bus(io = WindOff_A.power_out,
                    bus_io = bus_elec_A.input,
                    factor = 1., offset = 0.)
model.plug_to_bus(io = WindOn_A.power_out,
                    bus_io = bus_elec_A.input,
                    factor = 1., offset = 0.)
model.plug_to_bus(io = hydroROR_A.power_out,
                    bus_io = bus_elec_A.input,
                    factor = 1., offset = 0.)

model.plug_to_bus(io = Solar_PV_B.power_out,
                    bus_io = bus_elec_B.input,
                    factor = 1., offset = 0.)
model.plug_to_bus(io = WindOff_B.power_out,
                    bus_io = bus_elec_B.input,
                    factor = 1., offset = 0.)
model.plug_to_bus(io = WindOn_B.power_out,
                    bus_io = bus_elec_B.input,
                    factor = 1., offset = 0.)
model.plug_to_bus(io = hydroROR_B.power_out,
                    bus_io = bus_elec_A.input,
                    factor = 1., offset = 0.)

#### Model links

In [27]:
# if global_bool_CCGT_cluster:
#     model.add_link(io_1=cluster_CCGT_A.n_machine,
#                    io_2=bus_gas_fopex.input,
#                    factor=1., offset=0.)
#     model.add_link(io_1=cluster_CCGT_B.n_machine,
#                    io_2=bus_gas_fopex.output,
#                    factor=1., offset=0.)
# # end if

## Simulation

In [28]:
model.solve({SolverOption.MILP_GAP : 0.02, SolverOption.WRITE_PROBLEM:True, SolverOption.PRINT_LOG:True})

Running first time step
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                6
CPXPARAM_Threads                                 8
CPXPARAM_RandomSeed                              201804051
CPXPARAM_MIP_Strategy_StartAlgorithm             6
CPXPARAM_MIP_Tolerances_MIPGap                   0.02
Found incumbent of value 3.9821426e+10 after 0.03 sec. (8.86 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 21322 rows and 29759 columns.
MIP Presolve modified 3588 coefficients.
Aggregator did 7773 substitutions.
Reduced MIP has 20825 rows, 25472 columns, and 161103 nonzeros.
Reduced MIP has 11700 binaries, 200 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.29 sec. (209.65 ticks)
Probing fixed 0 vars, tightened 539 bounds.
Probing time = 0.11 sec. (15.47 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 4419 rows and 4378 columns.
Reduced MIP has 16406 rows, 21094 columns, and 144802 nonzeros.
Reduced MIP has 11700 binaries, 20

/home/trilogy-public/Python-3.7.9/venv/lib/python3.7/site-packages/pandas/core/indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = value


Running provided post-processing function.
Running time step 2
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                6
CPXPARAM_Threads                                 8
CPXPARAM_RandomSeed                              201804051
CPXPARAM_MIP_Strategy_StartAlgorithm             6
CPXPARAM_MIP_Tolerances_MIPGap                   0.02
Tried aggregator 2 times.
MIP Presolve eliminated 21563 rows and 29936 columns.
MIP Presolve modified 3705 coefficients.
Aggregator did 7723 substitutions.
Reduced MIP has 20716 rows, 25345 columns, and 158870 nonzeros.
Reduced MIP has 11700 binaries, 200 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.65 sec. (230.98 ticks)
Found incumbent of value 3.6555076e+10 after 0.87 sec. (306.15 ticks)
Probing fixed 0 vars, tightened 485 bounds.
Probing time = 0.47 sec. (15.81 ticks)
Cover probing fixed 4 vars, tightened 200 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 4906 rows and 5116 columns.
Reduced

/home/trilogy-public/Python-3.7.9/venv/lib/python3.7/site-packages/pandas/core/indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = value


Running provided post-processing function.
Running time step 3
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                6
CPXPARAM_Threads                                 8
CPXPARAM_RandomSeed                              201804051
CPXPARAM_MIP_Strategy_StartAlgorithm             6
CPXPARAM_MIP_Tolerances_MIPGap                   0.02
Tried aggregator 2 times.
MIP Presolve eliminated 21824 rows and 30183 columns.
MIP Presolve modified 3705 coefficients.
Aggregator did 7696 substitutions.
Reduced MIP has 20482 rows, 25125 columns, and 158035 nonzeros.
Reduced MIP has 11700 binaries, 200 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.74 sec. (366.84 ticks)
Found incumbent of value 2.4029483e+10 after 0.90 sec. (441.18 ticks)
Probing fixed 0 vars, tightened 503 bounds.
Probing time = 0.57 sec. (16.09 ticks)
Cover probing fixed 3 vars, tightened 182 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 4777 rows and 5014 columns.
Reduced

...

/home/trilogy-public/Python-3.7.9/venv/lib/python3.7/site-packages/pandas/core/indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = value


Running provided post-processing function.
Running time step 11
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                6
CPXPARAM_Threads                                 8
CPXPARAM_RandomSeed                              201804051
CPXPARAM_MIP_Strategy_StartAlgorithm             6
CPXPARAM_MIP_Tolerances_MIPGap                   0.02
Tried aggregator 2 times.
MIP Presolve eliminated 16184 rows and 22773 columns.
MIP Presolve modified 7083 coefficients.
Aggregator did 11479 substitutions.
Reduced MIP has 22339 rows, 28752 columns, and 201899 nonzeros.
Reduced MIP has 14964 binaries, 200 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.72 sec. (268.13 ticks)
Found incumbent of value 3.8790199e+10 after 0.94 sec. (355.39 ticks)
Probing fixed 0 vars, tightened 569 bounds.
Probing time = 0.51 sec. (20.41 ticks)
Cover probing fixed 13 vars, tightened 347 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 3641 rows and 3682 columns.
Redu

/home/trilogy-public/Python-3.7.9/venv/lib/python3.7/site-packages/pandas/core/indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = value


Running provided post-processing function.
Running time step 12
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                6
CPXPARAM_Threads                                 8
CPXPARAM_RandomSeed                              201804051
CPXPARAM_MIP_Strategy_StartAlgorithm             6
CPXPARAM_MIP_Tolerances_MIPGap                   0.02
Row 'c24012' infeasible, all entries at implied bounds.
Presolve time = 0.09 sec. (21.71 ticks)

Root node processing (before b&c):
  Real time             =    0.12 sec. (32.75 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.12 sec. (32.75 ticks)


UnsolvableProblemException: The given problem is not solvable.

In [ ]:
results = model.get_results(as_dataframe=False)

In [ ]:
if True:
    print(results.keys())

    # if global_bool_Nuke_indiv:
    #     print(results["flexible_static_Nuke"].keys())
    # # end if

    # if global_bool_Nuke_cluster:
    #     print(results["cluster_Nuke"].keys())
    # # end if

    # if global_bool_CCGT_cluster:
    #     print(results["cluster_CCGT"].keys())
    # # end if
# end if True to collapse the bloc

## Displaying results

### Plotting the results

#### Plotting parameters

In [ ]:
x_lb = 0
x_ub = global_step_count
grid_x_ticks = np.arange(x_lb, x_ub, 100)

print_ELPO_count = False

# x_lb = 1800
# x_ub = 1900
# grid_x_ticks = np.arange(x_lb, x_ub, 1)

#### Global plots

In [ ]:
if True:
    fig, ((ax11), (ax21), (ax31)) = plt.subplots(3, 1, figsize=(20,25))
    # grid_x_ticks = np.arange(x_lb, x_ub, 1)


    for ax in [ax11, ax21, ax31]:
        ax.set_xlim(x_lb, x_ub)
        ax.set_xticks(grid_x_ticks , minor=True)
        ax.grid(which="both")
    # end for x axis bounding

    ################################################################################################
    if True:
        ax11.title.set_text("Production profile and load on both buses")
        
        ax11.set_ylabel("Power (MW)")

        ax11.plot(results["load_A"]["power_in"], label = "Load_A", color="green")
        ax11.plot(results["load_B"]["power_in"], label = "Load_B", color="orange")

        ax11.plot(results["Interco_AB"]["power_out"], linestyle="dashed", label = "Interco_AB", color="orange")
        ax11.plot(results["Interco_BA"]["power_out"], linestyle="dashed", label = "Interco_BA", color="green")

        ax11.set_xlabel("Time in hours")
        ax11.legend()
    # end if True for graph (1,1)
    ################################################################################################
    if True:
        ax21.title.set_text("Balancing loads: unsupplied energy")

        ax21.set_xlabel("Time in hours")
        ax21.set_ylabel("Power (MW)")
        
        ax21.plot(results["load_A"]["power_in"], label = "Load_A", color="green")
        ax21.plot(results["load_B"]["power_in"], label = "Load_B", color="orange")

        if global_bool_virtual_balancing:
            ax21.plot(results["Demand_side_Management_A"]["power_out"], label = "DSM_A", linestyle = "dotted", color="green")

            ax21.plot(results["Demand_side_Management_B"]["power_out"], label = "DSM_B", linestyle = "dotted", color="orange")
        # end if

        ax21.legend()
    # end if True for graph (2,1)
    ################################################################################################
    if True:
        ax31.title.set_text("Balancing loads: spilled energy")

        ax31.set_xlabel("Time in hours")
        ax31.set_ylabel("Power (MW)")
        
        ax31.plot(results["load_A"]["power_in"], label = "Load_A", color="green")
        ax31.plot(results["load_B"]["power_in"], label = "Load_B", color="orange")

        if global_bool_virtual_balancing:
            ax31.plot(results["spilled_energy_A"]["power_in"], label = "Spilled_A", linestyle = "dashed", color="yellow")

            ax31.plot(results["spilled_energy_B"]["power_in"], label = "Spilled_B", linestyle = "dashed", color="blue")
        # end if

        ax31.legend()
    # end if True for graph (3,1)

    plt.tight_layout()
    plt.savefig(fname="Results.png") 
    plt.show()
# end if True

#### Results by plant

##### Flexible nuclear plants in zone A

In [ ]:
if global_bool_Nuke_indiv:
    if True:
        for i in range(Flex_nuke_n_A):
            key_unit = "Flexible_Nuke_A_"+str(i)
            key_ratio = "ratio_p_min_"+str(i)
        # end for
            
        if print_ELPO_count:
            nb_col = 3
        else:
            nb_col = 2
        # end if
        
        time = range(global_step_count)
    # end if True

    fig, axs = plt.subplots(Flex_nuke_n_A, nb_col, figsize=(20,7.5 * Flex_nuke_n_A))

    if True:
        if print_ELPO_count:
            for ax0, ax1, ax2 in axs:
                ax0.set_xlim(x_lb, x_ub)
                ax1.set_xlim(x_lb, x_ub)
                ax1.set_xlim(x_lb, x_ub)
            # end for
        else:
            for ax0, ax1 in axs:
                ax0.set_xlim(x_lb, x_ub)
                ax1.set_xlim(x_lb, x_ub)
            # end for
    # end if True for x-axis bounds definition

    for row in range(Flex_nuke_n_A):
        key_unit = "Flexible_Nuke_A_"+str(row)
        ratio_p_min = [0 for i in range(global_step_count)]

        if True:
            for i in range(global_step_count):
                burn_up = results[key_unit]["fpd"][i]/NPP_cycle_duration_fpd
                if burn_up <= 0.1:
                    ratio_p_min[i] = 100
                elif burn_up <= 0.65:
                    ratio_p_min[i] = 0.2*100
                elif burn_up <= 0.9:
                    ratio_p_min[i] = (0.2 + (burn_up-0.65) * (0.86-0.2)/(0.9-0.65))*100
                else:
                    ratio_p_min[i] = 0.86*100

                if results[key_unit]["RS"][i] == 1:
                    ratio_p_min[i] = 0
            # end for

            axs[row, 0].title.set_text(f"Nuclear production level of {key_unit} and minimum level jig evolving with burn-up")

            axs[row, 0].set_xlabel("Time in hours")
            axs[row, 0].set_ylabel("Relative power (%Pn)")

            axs[row, 0].plot(results[key_unit]["electricity"]/NPP_P_max*100, label = key_unit+" power out", color="r")
            axs[row, 0].plot(ratio_p_min, label = "p_min jig", color="b", linestyle="dashed", alpha=.5)
            axs[row,0].fill_between(time, ratio_p_min, facecolor="b", alpha=.2)

            ax0bis = axs[row, 0].twinx()
            ax0bis.set_ylabel("Burn-up")

            ax0bis.plot(results[key_unit]["fpd"]/NPP_cycle_duration_fpd, label = key_unit+" burn_up", color="green")
            
            axs[row, 0].legend()
            ax0bis.legend()
        # end if True for first column plots
        
        if True:
            axs[row, 1].title.set_text(f"{key_unit} state and modulation credit")
            
            axs[row, 1].set_xlabel("Time in hours")
            axs[row, 1].set_ylim(5, 55)
            
            labels = ["Refuelling stop", "Cold shutdown", "Hot standby", "Low-power Ops", "Power Ops"]
            colors = ["purple", "b", "r", "y", "g"]
            axs[row, 1].set_yticks([10,20,30,40,50], labels)
            axs[row, 1].tick_params(axis="y")

            ax1bis = axs[row, 1].twinx()
            ax1bis.set_ylabel("Remaining modulation credit")
            ax1bis.tick_params(axis="y")
            ax1bis.set_ylim(0,220)

            state = 10*results[key_unit]["RS"] + 20*results[key_unit]["CS"] + 30*results[key_unit]["HS"] + 40*results[key_unit]["LPO"] + 50*results[key_unit]["PO"]
            # Setting colors for better understanding of states
            statec = []
            for t in range(len(state)):
                statec.append(colors[int(state[t]//10)-1])
            # end for
                
            axs[row, 1].scatter(x = time, y=state, c=statec, marker=".", label = "flexible_static_Nuke state")

            ax1bis.plot(results[key_unit]["creditELPO"], color = "black", label = "flexible_static_Nuke crédit K")

            axs[row, 1].fill_between(time, 10, 0, where=(state>=10), facecolor="purple", alpha=.1)
            axs[row, 1].fill_between(time, 20, 10, where=(state>=20), facecolor="b", alpha=.1)
            axs[row, 1].fill_between(time, 30, 20, where=(state>=30), facecolor="r", alpha=.1)
            axs[row, 1].fill_between(time, 40, 30, where=(state>=40), facecolor="y", alpha=.1)
            axs[row, 1].fill_between(time, 50, 40, where=(state>=50), facecolor="g", alpha=.1)

            axs[row, 1].legend()
            ax1bis.legend()
        # end if True for second column plots

        if print_ELPO_count:
            axs[row, 2].title.set_text("Extended low-power operations monitoring")

            axs[row, 2].set_ylim(0, 25)

            axs[row, 2].set_xlabel("Time in hours")
            axs[row, 2].set_ylabel("LPO counter on past 24 hours")

            labels = ["", "ELPO threshold", ""]

            ax2bis = axs[row, 2].twinx()
            ax2bis.set_ylim(0, 24)
            ax2bis.set_yticks([0,8,25], labels)
            ax2bis.tick_params(axis="y")

            if global_NPP_bool_creditELPO:
                count_LPO = results[key_unit]["countLPO"]
                trimmed_LPO_during_ELPO = results[key_unit]["countLPO"] * results[key_unit]["is_step_ELPO"]

                axs[row, 2].plot(count_LPO, label = "flexible_Nuke countLPO")
                ax2bis.plot(trimmed_LPO_during_ELPO, label = "flexible_Nuke is_step_ELPO", color="orange")
            
                ax2bis.fill_between(time, 8, 0, facecolor="blue", alpha=.1)
                ax2bis.fill_between(time, 24, 8, facecolor="orange", alpha=.1)
            # end if
            
            axs[row, 2].legend()
        # end if for third column plots
    # end for

    plt.tight_layout()   
    plt.show()
# end if True

#### Clusters gaz

In [ ]:
if True:
    time = range(global_step_count)
# end if True

fig, axs = plt.subplots(2, 2, figsize=(20,15))

if True:
    for ax0, ax1 in axs:
        ax0.set_xlim(x_lb, x_ub)
        ax1.set_xlim(x_lb, x_ub)
    # end for
# end if True for x-axis bounds definition

for row in range(2):
    if row == 0:
        key_unit = "cluster_CCGT_A"
    elif row == 1:
        key_unit = "cluster_CCGT_B"
    # end if

    if True:
        axs[row, 0].title.set_text(f"Production level of {key_unit}")

        axs[row, 0].set_xlabel("Time in hours")
        axs[row, 0].set_ylabel("Relative power (%Pn)")

        axs[row, 0].plot(results[key_unit]["power_out"]/(CCGT_P_max*cluster_units[key_unit])*100, label = key_unit+" power out", color="r")

        axs[row, 0].legend()
    # end if True for first column plots
    
    if True:
        axs[row, 1].title.set_text(f"Number of units running in {key_unit}")
        
        axs[row, 1].set_xlabel("Time in hours")

        axs[row, 1].plot(results[key_unit]["n_machine"], color = "black", label = "number of machines running")

        axs[row, 1].legend()
    # end if True for second column plots
# end for

plt.tight_layout()   
plt.show()

#### Renewables

In [ ]:
if global_bool_CCGT_cluster:
    if True:
        time = range(global_step_count)
    # end if True

    fig, axs = plt.subplots(2, 1, figsize=(20,15))

    if True:
        for ax in axs:
            ax.set_xlim(x_lb, x_ub)
        # end for
    # end if True for x-axis bounds definition

    for row in range(2):
        if row == 0:
            key_unit = "A"
        elif row == 1:
            key_unit = "B"
        # end if

        if True:
            axs[row].title.set_text(f"Renewables production level in zone {key_unit}")

            axs[row].set_xlabel("Time in hours")
            axs[row].set_ylabel("Relative power (% of capacity installed)")

            for renewable in ["Solar_PV_"+key_unit, "WindOff_"+key_unit, "WindOn_"+key_unit]:
                axs[row].plot(results[renewable]["power_out"] / Renewables_capacities[renewable], label = renewable+" power out")

            axs[row].legend()
        # end if True for first column plots
    # end for

    plt.tight_layout()   
    plt.show()
# end if True

## Questions et sujets en cours

Pourquoi aucun usage des CCGT ?